# **Default Setting**

In [4]:
import tensorflow as tf
import cleverhans

import datetime
import json
import pprint

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from pathlib import Path

from evasion_attack.attack import AttackIdentificationModel, AttackVerificationModel, save_npz, load_npz
from evasion_attack.centroids import Centroids
from evasion_attack.checkpoints import Checkpoints
from evasion_attack.dataset import IdentificationDataLoader, VerificationDataLoader
from evasion_attack.evaluate import EvaluateIdentificationModel, EvaluateVerificationModel

from evasion_attack.models.preprocess import Preprocessing
from evasion_attack.models.resnet import embedding_model
from evasion_attack.models.trainer import AngularPrototypicalModel

print("VERSION:")
print(f"TensorFlow: {tf.__version__}")
print(f"CleverHans: {cleverhans.__version__}")

VERSION:
TensorFlow: 2.5.0
CleverHans: 4.0.0-edc15c6ec93c96562523dc42ae33c9e7


In [2]:
!nvidia-smi; free -h

Sun Aug 22 18:02:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:0A:00.0  On |                  N/A |
|  0%   41C    P8    25W / 220W |    360MiB /  7979MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
class LoadConfig:
    def __init__(self, conf: dict):
        for key, value in conf.items():
            setattr(self, key, value)
            
with open("config.json", "r") as f:
    config = LoadConfig(json.load(f))
    
config.attack_type = ["fgm", "pgd"]
config.epsilon = [1e-3, 1e-2, 1e-1]

In [23]:
def display(config):
    pprint.PrettyPrinter(indent=4).pprint(config)

display(vars(config))

{   'alpha': 0.05,
    'attack_type': ['fgm', 'pgd'],
    'buffer_size': 150000,
    'checkpoint_callback': True,
    'ckpt_dir': 'ckpt',
    'clear_assets': True,
    'csv_logger_callback': True,
    'data_path': 'data',
    'embedding_dim': 512,
    'epochs': 80,
    'epsilon': [0.001, 0.01, 0.1],
    'file_num_per_tfrecord': 5000,
    'global_batch_size': 64,
    'iden_tfrec_folder': 'data/tfrecord/iden',
    'init_lr': 0.001,
    'learning_rate_schedular_callback': True,
    'log_dir': 'logs',
    'model_name': 'AngularPrototypicalModel-Identification-20210822-170529',
    'model_type': 'iden',
    'num_classes_for_iden': 1251,
    'num_classes_for_veri': 1211,
    'num_iden_ts_ds': 8251,
    'num_slice': 10,
    'num_veri_ts_ds': 37720,
    'rectify': True,
    'result_path': 'result',
    'sample_rate': 16000,
    'seed': 42,
    'slice_len_sec': 2,
    'tensorboard_callback': True,
    'tr_folder': 'data/vox1_dev_wav/wav',
    'train_model': False,
    'ts_folder': 'data/vox1_te

# **Load Dataset**

In [24]:
def build_dataset(config, model_type, random_slice, global_batch_size):
    """ Build dataset for identification or verification task tricky.
    """
    assert config.model_type.lower() in ["iden", "veri"]

    def _build_iden_dataset():
        """ Build dataset for identification task.
        """
        ## Load tfrecords and build dataset.
        ts_filenames = sorted(list(Path(config.iden_tfrec_folder).glob("ts_*.tfrec")))

        ts_ds = IdentificationDataLoader.get_dataset(
            tfrecord_filenames=ts_filenames,
            cache=False,
            repeats=False,
            random_slice=random_slice,
            slice_len=config.slice_len_sec * config.sample_rate,
            num_slice=config.num_slice,
            shuffle=False,
            buffer_size=config.buffer_size,
            global_batch_size=global_batch_size,
        )

        return ts_ds
    
    def _build_veri_dataset():
        """ Build dataset for verification task.
        """
        ## Load tfrecords and build dataset.
        ts_filenames = sorted(list(Path(config.veri_tfrec_folder).glob("ts_*.tfrec")))

        ts_ds = VerificationDataLoader.get_dataset(
            tfrecord_filenames=ts_filenames,
            cache=False,
            repeats=False,
            random_slice=random_slice,
            slice_len=config.slice_len_sec * config.sample_rate,
            num_slice=config.num_slice,
            shuffle=False,
            buffer_size=config.buffer_size,
            global_batch_size=global_batch_size,
        )

        return ts_ds

    ts_ds = _build_iden_dataset() if model_type == "iden" else _build_veri_dataset()

    ## Priht the shapes.
    print(f"ts_ds: {ts_ds}")

    return ts_ds

In [25]:
## Sample.
ts_ds = build_dataset(config, model_type="iden", random_slice=True, global_batch_size=config.global_batch_size)

ts_ds: <PrefetchDataset shapes: ((None, None), (None,)), types: (tf.float32, tf.int64)>


# **Restore the Latest Model**

## **Build Model**

In [26]:
def build_model(config):
    sample_rate_ms = int(config.sample_rate / 1_000)
    num_classes = config.num_classes_for_iden if config.model_type.lower() == "iden" else config.num_classes_for_veri

    assert config.model_type in ["iden", "veri"]
    if config.model_type == "iden":
        config.model_name = "AngularPrototypicalModel-Identification"
    else:
        config.model_name = "AngularPrototypicalModel-Verification"
    config.model_name = f"{config.model_name}-{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"

    ## Define the parts.
    header = Preprocessing(
        frame_length=25 * sample_rate_ms,
        frame_step=10 * sample_rate_ms,
        fft_length=512,
        pad_end=True,
        num_mel_bins=64,
        sample_rate=config.sample_rate,
        lower_edge_hertz=0.,
        upper_edge_hertz=8_000.,
    )
    emb_model = embedding_model(
        input_shape=(config.slice_len_sec * config.sample_rate,),
        num_classes=num_classes,
        embedding_dim=config.embedding_dim,
        preprocessing_fn=header,
    )
    centroids = Centroids(
        num_classes=num_classes,
        embedding_dim=config.embedding_dim,
    )
    model = AngularPrototypicalModel(
        embedding_model=emb_model,
        centroids=centroids,
        name=config.model_name,
    )

    return model

In [27]:
model = build_model(config)

model.build([config.global_batch_size, config.slice_len_sec * config.sample_rate])
model.summary()

Model: "AngularPrototypicalModel-Identification-20210822-180440"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Functional)         (None, 512)               7440985   
Total params: 8,082,750
Trainable params: 7,432,219
Non-trainable params: 650,531
_________________________________________________________________


## **Restore Latest Checkpoint**

In [28]:
def get_latest_model(config, model: tf.keras.Model):
    """ Make clean and get latest model.
    """
    Checkpoints.make_clean(
        ckpt_dir=str(Path(config.ckpt_dir, config.model_type)),
    )
    latest_model = Checkpoints.load_latest_checkpoint(
        ckpt_dir=str(Path(config.ckpt_dir, config.model_type)), 
        model=model,
    )

    return latest_model

In [29]:
## Make clean checkpoints and load latest version.
latest_model = get_latest_model(config, model)

Checkpoint folder ckpt/iden is now clean, 0.00MB free.
Restored checkpoints: ckpt/iden/cp-067-0.2210.ckpt


# **Adversarial Attack**

In [14]:
iden_ts_filenames = sorted(list(Path(config.iden_tfrec_folder).glob("ts_*.tfrec")))
veri_ts_filenames = sorted(list(Path(config.veri_tfrec_folder).glob("ts_*.tfrec")))

totals = {
    "iden-random": int(np.ceil(sum([int(i.stem.split("_")[-1]) for i in iden_ts_filenames]) / config.global_batch_size)),
    "iden-fixed": config.num_iden_ts_ds,
    "veri-random": int(np.ceil(sum([int(i.stem.split("_")[-1]) for i in veri_ts_filenames]) / config.global_batch_size)),
    "veri-fixed": config.num_veri_ts_ds,
}

display(totals)

{   'iden-fixed': 8251,
    'iden-random': 129,
    'veri-fixed': 37720,
    'veri-random': 590}


## **Identification**

### **Random Sliced**

In [ ]:
ts_ds = build_dataset(config, model_type="iden", random_slice=True, global_batch_size=config.global_batch_size)

In [ ]:
%%time
for attack_type in config.attack_type:
    for epsilon in config.epsilon:
        assets = AttackIdentificationModel.attack_random_sliced_dataset(
            latest_model=latest_model,
            ds=ts_ds,
            attack_type=attack_type,
            epsilon=epsilon,
            total=totals["iden-random"],
        )
        save_npz(assets, save_dir=config.result_path)

### **Fixed Sliced**

In [24]:
ts_ds = build_dataset(config, model_type="iden", random_slice=False, global_batch_size=None)

ts_ds: <PrefetchDataset shapes: ((None,), ()), types: (tf.float32, tf.int64)>


In [ ]:
%%time
for attack_type in config.attack_type:
    for epsilon in config.epsilon:
        assets = AttackIdentificationModel.attack_fixed_sliced_dataset(
            latest_model=latest_model,
            ds=ts_ds,
            attack_type=attack_type,
            epsilon=epsilon,
            total=totals["iden-fixed"],
        )
        save_npz(assets, save_dir=config.result_path)

## **Verification**

### **Random Sliced**

In [24]:
ts_ds = build_dataset(config, model_type="veri", random_slice=True, global_batch_size=config.global_batch_size)

ts_ds: <PrefetchDataset shapes: ((None,), ()), types: (tf.float32, tf.int64)>


In [ ]:
%%time
for attack_type in config.attack_type:
    for epsilon in config.epsilon:
        assets = AttackVerificationModel.attack_random_sliced_dataset(
            latest_model=latest_model,
            ds=ts_ds,
            attack_type=attack_type,
            epsilon=epsilon,
            total=totals["veri-random"],
        )
        save_npz(assets, save_dir=config.result_path)

### **Fixed Sliced**

In [24]:
ts_ds = build_dataset(config, model_type="veri", random_slice=False, global_batch_size=None)

ts_ds: <PrefetchDataset shapes: ((None,), ()), types: (tf.float32, tf.int64)>


In [ ]:
# ## It may take too long time... :(
# %%time
# for attack_type in config.attack_type:
#     for epsilon in config.epsilon:
#         assets = AttackVerificationModel.attack_fixed_sliced_dataset(
#             latest_model=latest_model,
#             ds=ts_ds,
#             attack_type=attack_type,
#             epsilon=epsilon,
#             total=totals["veri-fixed"],
#         )
#         save_npz(assets, save_dir=config.result_path)

In [ ]:
%%time
for attack_type in list(config.attack_type[0]):
    for epsilon in config.epsilon:
        assets = AttackVerificationModel.attack_fixed_sliced_dataset(
            latest_model=latest_model,
            ds=ts_ds,
            attack_type=attack_type,
            epsilon=epsilon,
            total=totals["veri-fixed"],
        )
        save_npz(assets, save_dir=config.result_path)

In [ ]:
%%time
for attack_type in list(config.attack_type[1]):
    for epsilon in list(config.epsilon[0]):
        assets = AttackVerificationModel.attack_fixed_sliced_dataset(
            latest_model=latest_model,
            ds=ts_ds,
            attack_type=attack_type,
            epsilon=epsilon,
            total=totals["veri-fixed"],
        )
        save_npz(assets, save_dir=config.result_path)

In [ ]:
%%time
for attack_type in list(config.attack_type[1]):
    for epsilon in list(config.epsilon[1]):
        assets = AttackVerificationModel.attack_fixed_sliced_dataset(
            latest_model=latest_model,
            ds=ts_ds,
            attack_type=attack_type,
            epsilon=epsilon,
            total=totals["veri-fixed"],
        )
        save_npz(assets, save_dir=config.result_path)

In [ ]:
%%time
for attack_type in list(config.attack_type[2]):
    for epsilon in list(config.epsilon[0]):
        assets = AttackVerificationModel.attack_fixed_sliced_dataset(
            latest_model=latest_model,
            ds=ts_ds,
            attack_type=attack_type,
            epsilon=epsilon,
            total=totals["veri-fixed"],
        )
        save_npz(assets, save_dir=config.result_path)

# **Performance Evaluate**

## **Identification**

### **Random Sliced**

In [34]:
assets = load_npz(config.result_path, 'iden', 'random', None, None)
for attack_type in config.attack_type:
    for epsilon in config.epsilon:
        assets.update(load_npz(config.result_path, 'iden', 'random', attack_type, epsilon))

In [38]:
for file_name, asset in assets.items():
    result = EvaluateIdentificationModel.CMC(asset["y_true"], asset["y_pred"])
    print(f"{'[' + str(file_name) + ']':<35} Top-1 accuracy: {result[0]:.4f}, Top-5 accuracy: {result[4]:.4f}")

[result/iden-random-None-None.npz]  Top-1 accuracy: 0.7380, Top-5 accuracy: 0.8960
[result/iden-random-fgm-0.001.npz]  Top-1 accuracy: 0.4704, Top-5 accuracy: 0.7071
[result/iden-random-fgm-0.01.npz]   Top-1 accuracy: 0.1515, Top-5 accuracy: 0.3237
[result/iden-random-fgm-0.1.npz]    Top-1 accuracy: 0.0093, Top-5 accuracy: 0.0328
[result/iden-random-pgd-0.001.npz]  Top-1 accuracy: 0.0168, Top-5 accuracy: 0.0384
[result/iden-random-pgd-0.01.npz]   Top-1 accuracy: 0.0000, Top-5 accuracy: 0.0001
[result/iden-random-pgd-0.1.npz]    Top-1 accuracy: 0.0000, Top-5 accuracy: 0.0000


### **Fixed Sliced**

In [39]:
assets = load_npz(config.result_path, 'iden', 'fixed', None, None)
for attack_type in config.attack_type:
    for epsilon in config.epsilon:
        assets.update(load_npz(config.result_path, 'iden', 'fixed', attack_type, epsilon))

In [40]:
for file_name, asset in assets.items():
    result = EvaluateIdentificationModel.CMC(asset["y_true"], asset["y_pred"])
    print(f"{'[' + str(file_name) + ']':<35} Top-1 accuracy: {result[0]:.4f}, Top-5 accuracy: {result[4]:.4f}")

[result/iden-fixed-None-None.npz]   Top-1 accuracy: 0.8949, Top-5 accuracy: 0.9653
[result/iden-fixed-fgm-0.001.npz]   Top-1 accuracy: 0.7587, Top-5 accuracy: 0.9120
[result/iden-fixed-fgm-0.01.npz]    Top-1 accuracy: 0.2928, Top-5 accuracy: 0.5121
[result/iden-fixed-fgm-0.1.npz]     Top-1 accuracy: 0.0133, Top-5 accuracy: 0.0436
[result/iden-fixed-pgd-0.001.npz]   Top-1 accuracy: 0.0551, Top-5 accuracy: 0.0940
[result/iden-fixed-pgd-0.01.npz]    Top-1 accuracy: 0.0000, Top-5 accuracy: 0.0000
[result/iden-fixed-pgd-0.1.npz]     Top-1 accuracy: 0.0000, Top-5 accuracy: 0.0000


## **Verification**

### **Random Sliced**

In [41]:
assets = load_npz(config.result_path, 'veri', 'random', None, None)
for attack_type in config.attack_type:
    for epsilon in config.epsilon:
        assets.update(load_npz(config.result_path, 'veri', 'random', attack_type, epsilon))

In [42]:
for file_name, asset in assets.items():
    eer = EvaluateVerificationModel.EER(asset["y_true"], asset["y_pred"])
    auroc = EvaluateVerificationModel.AUROC(asset["y_true"], asset["y_pred"])
    mindcf = EvaluateVerificationModel.minDCF(asset["y_true"], asset["y_pred"])
    print(f"{'[' + str(file_name) + ']':<35} EER: {eer:.2f}, AUROC: {auroc:.4f}, minDCF: {mindcf:.4f}")

[result/veri-random-None-None.npz]  EER: 12.52, AUROC: 0.9484, minDCF: 0.6477
[result/veri-random-fgm-0.001.npz]  EER: 17.60, AUROC: 0.9052, minDCF: 0.8227
[result/veri-random-fgm-0.01.npz]   EER: 26.51, AUROC: 0.8140, minDCF: 0.9600
[result/veri-random-fgm-0.1.npz]    EER: 41.50, AUROC: 0.6180, minDCF: 0.9984
[result/veri-random-pgd-0.001.npz]  EER: 53.46, AUROC: 0.4545, minDCF: 0.9991
[result/veri-random-pgd-0.01.npz]   EER: 65.37, AUROC: 0.2927, minDCF: 1.0000
[result/veri-random-pgd-0.1.npz]    EER: 66.85, AUROC: 0.2705, minDCF: 1.0000


### **Fixed Sliced**

In [43]:
assets = load_npz(config.result_path, 'veri', 'fixed', None, None)
for attack_type in config.attack_type:
    for epsilon in config.epsilon:
        assets.update(load_npz(config.result_path, 'veri', 'fixed', attack_type, epsilon))

In [44]:
for file_name, asset in assets.items():
    eer = EvaluateVerificationModel.EER(asset["y_true"], asset["y_pred"])
    auroc = EvaluateVerificationModel.AUROC(asset["y_true"], asset["y_pred"])
    mindcf = EvaluateVerificationModel.minDCF(asset["y_true"], asset["y_pred"])
    print(f"{'[' + str(file_name) + ']':<35} EER: {eer:.2f}, AUROC: {auroc:.4f}, minDCF: {mindcf:.4f}")

[result/veri-fixed-None-None.npz]   EER: 6.55, AUROC: 0.9835, minDCF: 0.3974
[result/veri-fixed-fgm-0.001.npz]   EER: 8.93, AUROC: 0.9708, minDCF: 0.5302
[result/veri-fixed-fgm-0.01.npz]    EER: 18.11, AUROC: 0.8995, minDCF: 0.8161
[result/veri-fixed-fgm-0.1.npz]     EER: 37.61, AUROC: 0.6694, minDCF: 0.9987
[result/veri-fixed-pgd-0.001.npz]   EER: 58.78, AUROC: 0.3950, minDCF: 0.9937
[result/veri-fixed-pgd-0.01.npz]    EER: 80.59, AUROC: 0.1150, minDCF: 1.0000
[result/veri-fixed-pgd-0.1.npz]     EER: 82.89, AUROC: 0.0947, minDCF: 1.0000
